# Vehicle Detection and Tracking

**Vehicle Detection Project**

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.


**Generally, my approach in this project is starting and building on the top of the code supported in the project [lesson](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/2b62a1c3-e151-4a0e-b6b6-e424fa46ceab/lessons/fd66c083-4ccb-4fe3-bda1-c29db76f50a0/concepts/c3e815c7-1794-4854-8842-5d7b96276642)**

In [14]:
# Importing all required libraries
import numpy as np
import cv2
from skimage.feature import hog
import matplotlib.image as mpimg
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import pickle
import glob
import random
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, pause
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}
%matplotlib qt
print('end')

end


## Step1: Data Loading and Exploration

In this step, I will do the following:
1. Load data
2. suffle
3. Normalize
4. split into training and test data

In [15]:
# Loading data
non_vehicles = glob.glob('./non-vehicles/**/*.png')
vehicles = glob.glob('./vehicles/**/*.png')


# Display length of datasets
print('vehicle dataset length = ', len(vehicles))
print('non-vehicle dataset length = ', len(non_vehicles))



vehicle dataset length =  8792
non-vehicle dataset length =  8968


In [16]:
# Read in Random image and display shape
img = mpimg.imread(vehicles[10])
print ("a vehicle sample shape is ",img.shape)
#plt.imshow(img)
plt.imshow(img)
pause(1)


a vehicle sample shape is  (64, 64, 3)


**Display samples of the dataset**

In [17]:
# Visualize 20- random images from vehicle and non-vehicle dataset

fig, axs = plt.subplots(2,10, figsize=(15, 5))
fig.subplots_adjust(hspace = .05, wspace=.05)
fig.suptitle("visualize 20- random images from dataset", fontsize=16)
axs = axs.ravel()
for i in range(10):
    index = random.randint(0, len(vehicles))
   # image = cv2.imread(vehicles[index])
   # image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(cv2.imread(vehicles[index]), cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].imshow(img)
    axs[i].set_title('vehicle')
for i in range(10,20):
    index = random.randint(0, len(non_vehicles))
   # image = cv2.imread(vehicles[index])
   # image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(cv2.imread(non_vehicles[index]), cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].imshow(img)
    axs[i].set_title('non_vehicle')
    

    

## Step2: Features Extraction

In this step I will implement different methods for feature extraction, e.g. (Hog, color), the code used here is

In [18]:
# This function is used for coverting the image color space
# Note: This function is copied from the Udacity project lesson
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
print('end')

end


In [19]:
# This function is used for extracting Hog features
# Note: This function is copied from the Udacity project lesson
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features
print('end')

end


In [20]:
# This function is used for coverting the image color space
# Note: This function is copied from the Udacity project lesson
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
print('end')

end


In [21]:
# This function is used for coverting the image color space
# Note: This function is copied from the Udacity project lesson
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features
print('end')

end


**Explore Hog on vehicle images and non-vehicle images ** 

In [36]:
# Display Hog on a vehicle image and non-vehicle images
index = random.randint(0, len(vehicles))
v_img = mpimg.imread(vehicles[index]) #matplotlib read on scale from 0-->1
nv_img = mpimg.imread(non_vehicles[index])

orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 2 # Can be 0, 1, 2, or "ALL"

v_hog_features, v_hog_image = get_hog_features(v_img[:,:,hog_channel], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)
nv_hog_features, nv_hog_image = get_hog_features(nv_img[:,:,hog_channel], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)

fig, axs = plt.subplots(2,2, figsize=(15, 5))
fig.subplots_adjust(hspace = .2, wspace=.005)
fig.suptitle("HOG_exploration", fontsize=16)
axs = axs.ravel()
axs[0].axis('off')
axs[0].imshow(v_img)
axs[0].set_title('Car')
axs[1].axis('off')
axs[1].imshow(v_hog_image)
axs[1].set_title('Car-Hog')
axs[2].axis('off')
axs[2].imshow(nv_img)
axs[2].set_title('Not-Car')
axs[3].axis('off')
axs[3].imshow(nv_hog_image)
axs[3].set_title('Not-Car-Hog')

C:\Users\Noura\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Text(0.5,1,'Not-Car-Hog')

## Extract hog features from a list of Images

In [22]:

# Note: This function is copied from the Udacity project lesson
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

print('end')

end


## Test on a data sample of smaller size and Normalizing (Scaling)

In [39]:
# Reduce the sample size because HOG features are slow to compute
sample_size = 500
cars = vehicles[0:sample_size]
notcars = non_vehicles[0:sample_size]
### TODO: Tweak these parameters and see how the results change.
colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

t=time.time()
car_features = extract_features(cars, cspace=colorspace, orient=orient,  pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
print('end')

C:\Users\Noura\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


17.82 Seconds to extract HOG features...
end


## Shuffle and split data between training and testing (20% for testing), then train a linear SVM network  and test.

In [40]:

# Note: This code  is copied from the Udacity project lesson
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
print('end')

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 5292
0.56 Seconds to train SVC...
Test Accuracy of SVC =  0.995
My SVC predicts:  [ 0.  0.  1.  1.  1.  1.  1.  0.  0.  0.]
For these 10 labels:  [ 0.  0.  1.  1.  1.  1.  1.  0.  0.  0.]
0.002 Seconds to predict 10 labels with SVC
end


Note: hog_channel can take values of 0, 1, 2, or "ALL", meaning that you extract HOG features from the first, second, third, or all color channels respectively.

### In this section, I will do some experiments by playing with parameters to get the better test accuracy

## Step3: Training the classifier.
Now, after getting the best feature extraction from the previous step, I will train the classifier on all the dataset.

In [ ]:

cars = vehicles[0:len(vehicles)]
notcars = non_vehicles[0:len(non_vehicles)]
### TODO: Tweak these parameters and see how the results change.
colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

t=time.time()
car_features = extract_features(cars, cspace=colorspace, orient=orient,  pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
print('end')

C:\Users\Noura\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
